# 再帰型ニューラルネットワーク

前のモジュールでは、テキストの豊かな意味表現を使用し、その埋め込みの上に単純な線形分類器を構築してきました。このアーキテクチャは、文中の単語の集約された意味を捉えることができますが、単語の**順序**を考慮していません。埋め込みの上での集約操作によって、元のテキストからこの情報が失われてしまうためです。このようなモデルは単語の順序をモデル化することができないため、テキスト生成や質問応答のようなより複雑で曖昧なタスクを解くことができません。

テキストシーケンスの意味を捉えるためには、**再帰型ニューラルネットワーク**（RNN）と呼ばれる別のニューラルネットワークアーキテクチャを使用する必要があります。RNNでは、文をネットワークに1つずつシンボルを通過させ、ネットワークはある**状態**を生成します。そして次のシンボルとともにその状態を再びネットワークに渡します。

与えられたトークンの入力シーケンス $X_0,\dots,X_n$ に対して、RNNはニューラルネットワークブロックのシーケンスを作成し、このシーケンスをバックプロパゲーションを使用してエンドツーエンドで学習します。各ネットワークブロックは入力として $(X_i,S_i)$ のペアを受け取り、結果として $S_{i+1}$ を生成します。最終状態 $S_n$ または出力 $X_n$ は線形分類器に渡され、結果を生成します。すべてのネットワークブロックは同じ重みを共有し、1回のバックプロパゲーションパスでエンドツーエンドで学習されます。

状態ベクトル $S_0,\dots,S_n$ がネットワークを通過するため、単語間の順序的な依存関係を学習することができます。例えば、シーケンスのどこかに単語 *not* が現れる場合、状態ベクトル内の特定の要素を否定することを学習し、結果として否定を表現することができます。

> 図中のすべてのRNNブロックの重みが共有されているため、同じ図を1つのブロック（右側）として表現することができ、再帰的なフィードバックループがネットワークの出力状態を入力に戻します。

では、再帰型ニューラルネットワークがニュースデータセットの分類にどのように役立つか見てみましょう。


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## シンプルなRNN分類器

シンプルなRNNの場合、各リカレントユニットは単純な線形ネットワークで構成されており、結合された入力ベクトルと状態ベクトルを受け取り、新しい状態ベクトルを生成します。PyTorchでは、このユニットを`RNNCell`クラスで表現し、そのようなセルのネットワークを`RNN`レイヤーとして表現します。

RNN分類器を定義するには、まず埋め込み層を適用して入力語彙の次元を下げ、その上にRNNレイヤーを配置します。


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Note:** ここでは簡単さを重視して未学習の埋め込み層を使用していますが、より良い結果を得るためには、前のユニットで説明したように、Word2VecやGloVe埋め込みを使用した事前学習済みの埋め込み層を利用することができます。より深く理解するために、このコードを事前学習済みの埋め込みに対応するように適応させることを検討してください。

今回の場合、パディングされたデータローダーを使用します。そのため、各バッチには同じ長さのパディングされたシーケンスが含まれます。RNN層は埋め込みテンソルのシーケンスを受け取り、次の2つの出力を生成します：
* $x$ は各ステップでのRNNセルの出力シーケンス
* $h$ はシーケンスの最後の要素に対する最終的な隠れ状態

その後、全結合の線形分類器を適用してクラス数を取得します。

> **Note:** RNNの訓練は非常に難しいです。RNNセルがシーケンスの長さに沿って展開されると、逆伝播に関与する層の数が非常に多くなります。そのため、小さい学習率を選択し、より大きなデータセットでネットワークを訓練する必要があります。良い結果を得るには時間がかかる可能性があるため、GPUの使用が推奨されます。


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## 長短期記憶（LSTM）

古典的なRNNの主な問題の1つは、いわゆる**勾配消失**問題です。RNNは1回の逆伝播でエンドツーエンドに学習するため、ネットワークの最初の層に誤差を伝播させるのが難しくなり、その結果、遠く離れたトークン間の関係を学習することができません。この問題を回避する方法の1つは、**ゲート**と呼ばれる仕組みを使用して**明示的な状態管理**を導入することです。この種のアーキテクチャで最もよく知られているものは、**長短期記憶（LSTM）**と**ゲート付きリレー単位（GRU）**です。

![長短期記憶セルの例を示す画像](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTMネットワークはRNNに似た構造で組織されていますが、層から層へ渡される2つの状態があります。それは、実際の状態$c$と隠れベクトル$h$です。各ユニットでは、隠れベクトル$h_i$が入力$x_i$と結合され、それらが**ゲート**を介して状態$c$に何が起こるかを制御します。各ゲートはシグモイド活性化関数（出力範囲は$[0,1]$）を持つニューラルネットワークであり、状態ベクトルと掛け算することでビットマスクのように考えることができます。以下のゲートがあります（上記の図で左から右へ）：
* **忘却ゲート**は隠れベクトルを受け取り、ベクトル$c$のどの成分を忘れるべきか、どの成分を通過させるべきかを決定します。
* **入力ゲート**は入力と隠れベクトルから情報を取り出し、それを状態に挿入します。
* **出力ゲート**は状態を$\tanh$活性化を持つ線形層を通して変換し、その後、隠れベクトル$h_i$を使用してその成分の一部を選択し、新しい状態$c_{i+1}$を生成します。

状態$c$の成分は、オン・オフを切り替えられるフラグのように考えることができます。例えば、シーケンス内で*Alice*という名前に出会ったとき、それが女性キャラクターを指していると仮定し、文中に女性名詞があるというフラグを状態に立てることができます。その後、*and Tom*というフレーズに出会ったときには、複数名詞があるというフラグを立てます。このようにして、状態を操作することで文の文法的な特性を追跡できると考えられます。

> **Note**: LSTMの内部構造を理解するための素晴らしいリソースとして、Christopher Olahによる[Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)という記事があります。

LSTMセルの内部構造は複雑に見えるかもしれませんが、PyTorchはこの実装を`LSTMCell`クラス内に隠しており、LSTM層全体を表す`LSTM`オブジェクトを提供しています。そのため、LSTM分類器の実装は、前述の単純なRNNと非常に似たものになります。


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## パックされたシーケンス

この例では、ミニバッチ内のすべてのシーケンスをゼロベクトルでパディングする必要がありました。この方法ではメモリの無駄が発生するだけでなく、RNNではパディングされた入力項目のために追加のRNNセルが作成されることが問題となります。これらのセルは学習に参加しますが、重要な入力情報を持っていません。そのため、RNNを実際のシーケンスサイズに合わせて学習させる方がはるかに効率的です。

これを実現するために、PyTorchではパディングされたシーケンスを特別な形式で保存する方法が導入されています。例えば、以下のようなパディングされたミニバッチがあるとします：
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
ここで、0はパディングされた値を表し、入力シーケンスの実際の長さのベクトルは `[5,3,1]` です。

パディングされたシーケンスでRNNを効率的に学習させるためには、まず最初のRNNセルのグループを大きなミニバッチ（`[1,6,9]`）で学習を開始し、次に3番目のシーケンスの処理を終了して、短縮されたミニバッチ（`[2,7]`、`[3,8]`）で学習を続ける必要があります。このようにして、パックされたシーケンスは1つのベクトルとして表されます。この場合、`[1,6,9,2,7,3,8,4,5]` と長さのベクトル（`[5,3,1]`）で表現され、これを使って元のパディングされたミニバッチを簡単に再構築することができます。

パックされたシーケンスを生成するには、`torch.nn.utils.rnn.pack_padded_sequence` 関数を使用します。RNN、LSTM、GRUを含むすべての再帰層は、入力としてパックされたシーケンスをサポートしており、パックされた出力を生成します。この出力は、`torch.nn.utils.rnn.pad_packed_sequence` を使用してデコードできます。

パックされたシーケンスを生成するには、ネットワークに長さのベクトルを渡す必要があるため、ミニバッチを準備するための別の関数が必要です。


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

実際のネットワークは上記の `LSTMClassifier` に非常に似ていますが、`forward` パスではパディングされたミニバッチとシーケンス長のベクトルの両方を受け取ります。埋め込みを計算した後、パックされたシーケンスを計算し、それを LSTM レイヤーに渡し、結果を再びアンパックします。

> **Note**: 実際にはアンパックされた結果 `x` を使用しません。なぜなら、後続の計算では隠れ層からの出力を使用するためです。したがって、このコードからアンパック処理を完全に削除することも可能です。ここにアンパック処理を残している理由は、もしネットワークの出力をさらに計算で使用する必要がある場合に、このコードを簡単に修正できるようにするためです。


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **注意:** トレーニング関数に渡すパラメータ `use_pack_sequence` に気付いたかもしれません。現在、`pack_padded_sequence` 関数は長さシーケンステンソルがCPUデバイス上にあることを要求しているため、トレーニング関数はトレーニング時に長さシーケンスデータをGPUに移動することを避ける必要があります。[`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py) ファイル内の `train_emb` 関数の実装を確認することができます。


## 双方向および多層RNN

これまでの例では、すべてのリカレントネットワークがシーケンスの始まりから終わりに向かって一方向に動作していました。それは自然に感じられます。なぜなら、私たちが文章を読んだり音声を聞いたりする方法に似ているからです。しかし、実際のケースでは入力シーケンスにランダムアクセスできることが多いため、リカレント計算を両方向で実行する方が理にかなっている場合があります。このようなネットワークは**双方向**RNNと呼ばれ、RNN/LSTM/GRUのコンストラクタに`bidirectional=True`パラメータを渡すことで作成できます。

双方向ネットワークを扱う場合、各方向に対して1つずつ、2つの隠れ状態ベクトルが必要になります。PyTorchではこれらのベクトルを2倍のサイズの1つのベクトルとしてエンコードします。これは非常に便利です。なぜなら、通常は結果として得られる隠れ状態を全結合の線形層に渡すため、このサイズの増加を考慮して層を作成するだけで済むからです。

リカレントネットワーク（単方向でも双方向でも）は、シーケンス内の特定のパターンを捉え、それを状態ベクトルに保存したり、出力に渡したりすることができます。畳み込みネットワークと同様に、最初の層によって抽出された低レベルのパターンを基に、より高次のパターンを捉えるために、もう1つのリカレント層をその上に構築することができます。これにより、**多層RNN**という概念が生まれます。これは2つ以上のリカレントネットワークで構成され、前の層の出力が次の層の入力として渡されます。

![多層長短期記憶RNNを示す画像](../../../../../translated_images/ja/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Fernando Lópezによる[素晴らしい投稿](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3)からの画像*

PyTorchでは、このようなネットワークの構築が簡単です。RNN/LSTM/GRUのコンストラクタに`num_layers`パラメータを渡すだけで、複数のリカレント層を自動的に構築できます。この場合、隠れ状態ベクトルのサイズも比例して増加するため、リカレント層の出力を処理する際にこの点を考慮する必要があります。


## 他のタスクにおけるRNN

このユニットでは、RNNがシーケンス分類に使用できることを学びましたが、実際にはテキスト生成や機械翻訳など、さらに多くのタスクを処理することができます。これらのタスクについては次のユニットで検討します。



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は一切の責任を負いません。
